# MassMutual People Counter

In [5]:
from random import randint
import numpy as np
import cv2
import time


class MyPerson:
    tracks = []

    def __init__(self, i, xi, yi, max_age):
        self.i = i
        self.x = xi
        self.y = yi
        self.tracks = []
        self.R = randint(0, 255)
        self.G = randint(0, 255)
        self.B = randint(0, 255)
        self.done = False
        self.state = '0'
        self.age = 0
        self.max_age = max_age
        self.dir = None

    def getRGB(self):
        return (self.R, self.G, self.B)

    def getTracks(self):
        return self.tracks

    def getId(self):
        return self.i

    def getState(self):
        return self.state

    def getDir(self):
        return self.dir

    def getX(self):
        return self.x

    def getY(self):
        return self.y

    def updateCoords(self, xn, yn):
        self.age = 0
        self.tracks.append([self.x, self.y])
        self.x = xn
        self.y = yn

    def setDone(self):
        self.done = True

    def timedOut(self):
        return self.done

    def going_UP(self, mid_start, mid_end):
        if len(self.tracks) >= 2:
            if self.state == '0':
                if self.tracks[-1][1] < mid_end and self.tracks[-2][1] >= mid_end:                       #noqa
                    state = '1'
                    self.dir = 'up'
                    return True
            else:
                return False
        else:
            return False

    def going_DOWN(self, mid_start, mid_end):
        if len(self.tracks) >= 2:
            if self.state == '0':
                if self.tracks[-1][1] > mid_start and self.tracks[-2][1] <= mid_start:                   #noqa
                    state = '1'
                    self.dir = 'down'
                    return True
            else:
                return False
        else:
            return False

    def age_one(self):
        self.age += 1
        if self.age > self.max_age:
            self.done = True
        return True


class MultiPerson:
    def __init__(self, persons, xi, yi):
        self.persons = persons
        self.x = xi
        self.y = yi
        self.tracks = []
        self.R = randint(0, 255)
        self.G = randint(0, 255)
        self.B = randint(0, 255)
        self.done = False

In [7]:
import uuid
import json
import dweepy


run_id = str(uuid.uuid4())
run_platform = "Desktop Jupyter"
run_stamp = {
    "mess-type":"EXEC",
    "mess-format":"0.0.1",
    "run-id":run_id,
    "run-time":time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()),
    "run-status":"start",
    "run-platform":run_platform,
    "version":{
        "algorithm":"0.0.1",
        #"python":sys.version,
        #"ipython":IPthon.__version__,
        "cv2":cv2.__version__,
        #"jupyter":jupyter.__version__,
        #"numpy":numpy.__version__,
        #"imutils":imutils.__version__
    }
}

class trace_mess:
    y = 1
    
    def getY(self):
        return self.y

In [8]:
print(json.dumps(run_stamp))

# Input and Output Counters
cnt_up = 0
cnt_down = 0

# Video Source
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('videos/People-Walking-Shot-From-Above.mp4')

# Check if camera opened successfully
if (cap.isOpened() is False):
    print("Error opening video stream or file")
    print(json.dumps({"mess-type":"ERROR","run-id":run_id, "mess-text":"Could not open video stream or file"}))
    run_stamp['run-status'] = "stop"
    run_stamp['run-time'] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    print(json.dumps(run_stamp))
    exit

# Get current width and height of frame
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)     # float
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)   # float
print(json.dumps({"mess-type":"INFO","run-id":run_id, "frame":{"width":width, "height":height}}))

# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
fourcc = cv2.VideoWriter_fourcc('M','P','4','V')
fourcc = cv2.VideoWriter_fourcc(*'a\0\0\0')
out = cv2.VideoWriter('videos/output.mp4', fourcc, 20.0, (int(width), int(height)))                     #noqa

# Video properties
# cap.set(3, 160) # Width
# cap.set(4, 120) # Height

# Prints the capture properties to console
for i in range(19):
    print(i, cap.get(i))

w = cap.get(3)
h = cap.get(4)
frameArea = h*w
areaTH = frameArea/250
print('Area Threshold', areaTH)

# Input / output lines
line_up = int(2*(h/5))
line_down = int(3*(h/5))

up_limit = int(1*(h/5))
down_limit = int(4*(h/5))

print("Red line y:", str(line_down))
print("Blue line y:", str(line_up))
print(json.dumps({"mess-type":"INFO","run-id":run_id, "area threshold":areaTH, "lines":{"red y axis":str(line_down), "blue y axis":str(line_up)}}))

line_down_color = (255, 0, 0)
line_up_color = (0, 0, 255)
pt1 = [0, line_down]
pt2 = [w, line_down]
pts_L1 = np.array([pt1, pt2], np.int32)
pts_L1 = pts_L1.reshape((-1, 1, 2))
pt3 = [0, line_up]
pt4 = [w, line_up]
pts_L2 = np.array([pt3, pt4], np.int32)
pts_L2 = pts_L2.reshape((-1, 1, 2))

pt5 = [0, up_limit]
pt6 = [w, up_limit]
pts_L3 = np.array([pt5, pt6], np.int32)
pts_L3 = pts_L3.reshape((-1, 1, 2))
pt7 = [0, down_limit]
pt8 = [w, down_limit]
pts_L4 = np.array([pt7, pt8], np.int32)
pts_L4 = pts_L4.reshape((-1, 1, 2))

# Background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

# Structural elements for morphogic filters
kernelOp = np.ones((3, 3), np.uint8)
kernelOp2 = np.ones((5, 5), np.uint8)
kernelCl = np.ones((11, 11), np.uint8)

# Variables
font = cv2.FONT_HERSHEY_SIMPLEX
persons = []
max_p_age = 5
pid = 1

while(cap.isOpened()):
# for image in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):                 #noqa
    # Read an image from the video source
    ret, frame = cap.read()
#     frame = image.array

    for i in persons:
        i.age_one()   # age every person one frame
    #########################
    #   PRE-PROCESSING      #
    #########################

    # Apply subtraction of background
    fgmask = fgbg.apply(frame)
    fgmask2 = fgbg.apply(frame)

    # Binary to remove shadows (gray color)
    try:
        ret, imBin = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
        ret, imBin2 = cv2.threshold(fgmask2, 200, 255, cv2.THRESH_BINARY)
        # Opening (erode-> dilate) to remove noise
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask2 = cv2.morphologyEx(imBin2, cv2.MORPH_OPEN, kernelOp)
        # Closing (dilate -> erode) to join white regions
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernelCl)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        print('UP:', cnt_up)
        print('DOWN:', cnt_down)
        print(json.dumps({"mess-type":"INFO","run-id":run_id, "total count":{"up":cnt_up, "down":cnt_down}}))
        run_stamp['run-status'] = "stop"
        run_stamp['run-time'] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        print(json.dumps(run_stamp))
        break
    #################
    #    CONTOURS   #
    #################

    # RETR_EXTERNAL returns only extreme outer flags. All child contours are left behind.  #noqa
    _, contours0, hierarchy = cv2.findContours(mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)        #noqa
    for cnt in contours0:
        area = cv2.contourArea(cnt)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################

            # Missing add conditions for multipersons, outputs and screen inputs              #noqa

            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x, y, w, h = cv2.boundingRect(cnt)

            new = True
            if cy in range(up_limit, down_limit):
                for i in persons:
                    if abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h:
                        # the object is near one already detected before
                        new = False
                        i.updateCoords(cx, cy)   # update coordinates in the object and resets age     #noqa
                        if i.going_UP(line_down, line_up) is True:
                            cnt_up += 1
                            print("ID:", i.getId(), 'crossed going up at', time.strftime("%c"))          #noqa
                            print(json.dumps({"mess-type":"FEATURE","run-id":run_id, "object":{"id":i.getId(),"direction":"up","time":time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())}}))
                        elif i.going_DOWN(line_down, line_up) is True:
                            cnt_down += 1
                            print("ID:", i.getId(), 'crossed going down at', time.strftime("%c"))        #noqa
                            print(json.dumps({"mess-type":"FEATURE","run-id":run_id, "object":{"id":i.getId(),"direction":"down","time":time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())}}))
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                        elif i.getDir() == 'up' and i.getY() < up_limit:
                            i.setDone()
                    if i.timedOut():
                        # remove from the list persons
                        index = persons.index(i)
                        persons.pop(index)
                        del i     # free memory of i
                if new is True:
                    p = MyPerson(pid, cx, cy, max_p_age)
                    persons.append(p)
                    pid += 1
            #################
            #   DRAWINGS   #
            #################
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            img = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            # cv2.drawContours(frame, cnt, -1, (0, 255, 0), 3)

    # END for cnt in contours0

    #########################
    # DRAWING TRAJECTORIES  #
    #########################
    for i in persons:
        # if len(i.getTracks()) >= 2:
            # pts = np.array(i.getTracks(), np.int32)
            # pts = pts.reshape((-1, 1, 2))
            # frame = cv2.polylines(frame, [pts], False, i.getRGB())
        # if i.getId() == 9:
            # print(str(i.getX()), ', ', str(i.getY()))
        cv2.putText(frame, str(i.getId()), (i.getX(), i.getY()), font, 0.3, i.getRGB(), 1, cv2.LINE_AA)  #noqa

    #################
    #     IMAGES    #
    #################
    str_up = 'UP: ' + str(cnt_up)
    str_down = 'DOWN: ' + str(cnt_down)
    frame = cv2.polylines(frame, [pts_L1], False, line_down_color, thickness=2)
    frame = cv2.polylines(frame, [pts_L2], False, line_up_color, thickness=2)
    frame = cv2.polylines(frame, [pts_L3], False, (255, 255, 255), thickness=1)
    frame = cv2.polylines(frame, [pts_L4], False, (255, 255, 255), thickness=1)
    cv2.putText(frame, str_up, (10, 40), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)  #noqa
    cv2.putText(frame, str_up, (10, 40), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)  #noqa
    cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)  #noqa
    cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)  #noqa

    cv2.imshow('Frame', frame)
    # cv2.imshow('Mask', mask)

    # write the flipped frame
    out.write(frame)

    # preisonar ESC para salir
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
# END while(cap.isOpened())

    #################
    #   CLEANING    #
    #################
cap.release()
out.release()
cv2.destroyAllWindows()

{"run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "EXEC", "run-time": "2017-09-19 17:43:34", "mess-format": "0.0.1", "run-platform": "Desktop Jupyter", "version": {"cv2": "3.3.0", "algorithm": "0.0.1"}, "run-status": "start"}
{"run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "frame": {"height": 336.0, "width": 596.0}, "mess-type": "INFO"}
0 0.0
1 0.0
2 0.000333667000333667
3 596.0
4 336.0
5 29.97
6 828601953.0
7 758.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
Area Threshold 801.024
Red line y: 201
Blue line y: 134
{"run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "area threshold": 801.024, "mess-type": "INFO", "lines": {"blue y axis": "134", "red y axis": "201"}}
ID: 7 crossed going down at Tue 19 Sep 2017 01:43:39 PM EDT
{"object": {"time": "2017-09-19 17:43:39", "id": 7, "direction": "down"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 11 crossed going up at Tue 19 Sep 2017 01:43:39 PM EDT
{"object": {

ID: 55 crossed going up at Tue 19 Sep 2017 01:43:49 PM EDT
{"object": {"time": "2017-09-19 17:43:49", "id": 55, "direction": "up"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 59 crossed going down at Tue 19 Sep 2017 01:43:49 PM EDT
{"object": {"time": "2017-09-19 17:43:49", "id": 59, "direction": "down"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 51 crossed going down at Tue 19 Sep 2017 01:43:50 PM EDT
{"object": {"time": "2017-09-19 17:43:50", "id": 51, "direction": "down"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 51 crossed going down at Tue 19 Sep 2017 01:43:50 PM EDT
{"object": {"time": "2017-09-19 17:43:50", "id": 51, "direction": "down"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 51 crossed going down at Tue 19 Sep 2017 01:43:50 PM EDT
{"object": {"time": "2017-09-19 17:43:50", "id": 51, "direction": "down"}, "run-id": "d74c8940-4202-4ad

ID: 101 crossed going up at Tue 19 Sep 2017 01:44:03 PM EDT
{"object": {"time": "2017-09-19 17:44:03", "id": 101, "direction": "up"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 129 crossed going up at Tue 19 Sep 2017 01:44:04 PM EDT
{"object": {"time": "2017-09-19 17:44:04", "id": 129, "direction": "up"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 101 crossed going up at Tue 19 Sep 2017 01:44:05 PM EDT
{"object": {"time": "2017-09-19 17:44:05", "id": 101, "direction": "up"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 101 crossed going up at Tue 19 Sep 2017 01:44:06 PM EDT
{"object": {"time": "2017-09-19 17:44:06", "id": 101, "direction": "up"}, "run-id": "d74c8940-4202-4ada-989b-7ac929c37ec1", "mess-type": "FEATURE"}
ID: 101 crossed going down at Tue 19 Sep 2017 01:44:07 PM EDT
{"object": {"time": "2017-09-19 17:44:07", "id": 101, "direction": "down"}, "run-id": "d74c8940-4202-4ada-

## Show People Counting Video

<video controls src="videos/output.mp4" />

In [ ]:
%%HTML
<video width="596" height="336" controls>
  <source src="videos/output.mp4" type="video/mp4">

## Sources
* [People Counter blog by Federico Mejia Barajas](http://www.femb.com.mx/people-counter/people-counter-9-counting/)
* [Video of FEMB in action](https://www.youtube.com/watch?time_continue=9&v=aEcBnD80nLg)